In [4]:
import os
os.getcwd()

'C:\\Users\\W-HAWX\\PycharmProjects\\PDF_corpus_Collection'

In [5]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure, LTTextBox
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage, PDFTextExtractionNotAllowed
from pdfminer.pdfparser import PDFParser

In [6]:
from pathlib import Path
import xlsxwriter

encoding = 'ISO-8859-1'


def get_data_from_pdfs(number):
    path_pdfs = "PDF_Disaster_Risk_Reduction/" + str(number)

    pdf_text_paragraphs_list = []
    pdf_figures_stack = [""]
    paper_count = 0
    row = 0

    workbook = xlsxwriter.Workbook('dataset_' + str(number) + '.xlsx')

    worksheet = workbook.add_worksheet()

    for path in Path(path_pdfs).glob("*.pdf"):

        with path.open("rb") as f:
            parser = PDFParser(f)
            doc = PDFDocument(parser)
            page = list(PDFPage.create_pages(doc))[0]
            rsrcmgr = PDFResourceManager()
            #change laparams
            device = PDFPageAggregator(rsrcmgr, laparams=LAParams(line_margin=1.50, word_margin=0.01))
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            interpreter.process_page(page)
            layout = device.get_result()

            # debug
            print("PAPER: " + str(paper_count))

            for obj in layout:

                if isinstance(obj, LTTextBox):
                    # get each paragraph and append to a list  with lowercase
                    pdf_text_paragraphs_list.append(obj.get_text().lower())


                elif isinstance(obj, LTFigure):
                    pdf_figures_stack += list(obj)

            word_exist_abstract = "a b s t r a c t"

            keywords_data = ""
            
            abstract_data = ""
            
            # check the paragraph if it is abstract
            for paragraph in pdf_text_paragraphs_list:

                if word_exist_abstract in paragraph:
                    print("Found!")
                    abstract_paragraph = paragraph.replace(word_exist_abstract, "")
                    abstract_paragraph = abstract_paragraph.replace("\n", "")
                    
                #else:
                    #print("Not found!")
                    
            # print(pdf_text_paragraphs_list[abstract_index])
            # print("dict info :")
            # print(doc.info[0])
            abstract_data = abstract_paragraph

            dict_doc_info = doc.info[0]
            # retrieve title of the paper
            
            title_data = dict_doc_info["Title"].decode(encoding)
            # print(dict_doc_info["Title"].decode(encoding))

            # retrieve keywords from the paper
            try:
                keywords_data = dict_doc_info["Keywords"].decode(encoding)
            except:
                print("An exception occurred in keywords: ", title_data)
            # print(dict_doc_info["Keywords"].decode(encoding))

            print("Paper : "+ str(paper_count))
            print("Tittle : "+ title_data)
            print("keywords : "+ keywords_data)
            print("Abstract : "+ abstract_data)

            pdf_text_paragraphs_list.clear()

            # save on the Excel file dataset_papers

            titles_excel = ["Title", "Keywords", "Abstract", "Year", "PDF Directory"]
            content = [title_data, keywords_data, abstract_data, path_pdfs, str(path)]

            column = 0

            if row == 0:
                for item in titles_excel:
                    # write operation perform
                    worksheet.write(row, column, item)
                    column += 1
            else:
                # iterating through content list
                for item in content:
                    worksheet.write(row, column, item)
                    # incrementing the value of row by one
                    # with each iteration.
                    column += 1
            row += 1
        paper_count += 1

    workbook.close()


In [7]:
def begin_pdf_path_extract():
    for number in range(2017, 2022):
        get_data_from_pdfs(number)


In [ ]:
begin_pdf_path_extract()
